<a href="https://colab.research.google.com/github/keymc021/work/blob/master/least_squares_solution_from_gnss_derived_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
from pathlib import Path
datapath = Path("../input/google-smartphone-decimeter-challenge/")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Background

Every satellite in space continuously broadcasts its current position coordinate information to the world through satellite signals. Any GNSS receiver can receive and read this information through an antenna.

When the satellite sends position information, it also attaches the timestamp when the data was sent. After the GNSS receiver receives the data, it subtracts the time on the timestamp from the current time, which is the time taken for the data transmission. And the approximate distance between the satellite and the GPS receiver can be calculated by data transmission time multiplied by the transmission speed, which is the `rawPrM` in `_derived` table. 

The most difficult problem in GPS positioning is error. There are many reasons for positioning errors, such as the ionosphere, the receiving device, and the blocking and multipath effects. Inorder to get a more accurate the approximate distance between the satellite and the GPS receiver with as less error as possible, we can use a corrected approximate distance as [home page](https://www.kaggle.com/c/google-smartphone-decimeter-challenge/data) suggested, which can be computed as: `correctedPrM = rawPrM + satClkBiasM - isrbM - ionoDelayM - tropoDelayM`

<img src='https://www.researchgate.net/publication/324989880/figure/fig1/AS:623672356253699@1525706485241/Global-Navigation-Satellite-System-GNSS-spoofing-attack-illustration.png' width=400/>

Normally these values are need to be calculated carefully from GNSSLog file by lots of domain knowledge. Thanks to the dataset provider, we can directly use the intermediate calculated value from derived files to built the initial version of the model.

### Basic Utils Functions

In [ ]:
def ecef2lla(x, y, z):
  # x, y and z are scalars or vectors in meters
  x = np.array([x]).reshape(np.array([x]).shape[-1], 1)
  y = np.array([y]).reshape(np.array([y]).shape[-1], 1)
  z = np.array([z]).reshape(np.array([z]).shape[-1], 1)

  a=6378137
  a_sq=a**2
  e = 8.181919084261345e-2
  e_sq = 6.69437999014e-3

  f = 1/298.257223563
  b = a*(1-f)

  # calculations:
  r = np.sqrt(x**2 + y**2)
  ep_sq  = (a**2-b**2)/b**2
  ee = (a**2-b**2)
  f = (54*b**2)*(z**2)
  g = r**2 + (1 - e_sq)*(z**2) - e_sq*ee*2
  c = (e_sq**2)*f*r**2/(g**3)
  s = (1 + c + np.sqrt(c**2 + 2*c))**(1/3.)
  p = f/(3.*(g**2)*(s + (1./s) + 1)**2)
  q = np.sqrt(1 + 2*p*e_sq**2)
  r_0 = -(p*e_sq*r)/(1+q) + np.sqrt(0.5*(a**2)*(1+(1./q)) - p*(z**2)*(1-e_sq)/(q*(1+q)) - 0.5*p*(r**2))
  u = np.sqrt((r - e_sq*r_0)**2 + z**2)
  v = np.sqrt((r - e_sq*r_0)**2 + (1 - e_sq)*z**2)
  z_0 = (b**2)*z/(a*v)
  h = u*(1 - b**2/(a*v))
  phi = np.arctan((z + ep_sq*z_0)/r)
  lambd = np.arctan2(y, x)

  return phi*180/np.pi, lambd*180/np.pi, h

### Load Data

In [ ]:
df_baseline = pd.read_csv(datapath/"baseline_locations_train.csv")
df_sample_trail_gt = pd.read_csv(datapath/"train/2020-05-14-US-MTV-1/Pixel4/ground_truth.csv")
df_sample_trail = pd.read_csv(datapath/"train/2020-05-14-US-MTV-1/Pixel4/Pixel4_derived.csv")
df_sample_trail["correctedPrM"] = df_sample_trail["rawPrM"] + df_sample_trail["satClkBiasM"] - df_sample_trail["isrbM"] - df_sample_trail["ionoDelayM"] - df_sample_trail["tropoDelayM"] 

## Simple Least Square Solution

An intuitive solution to solve this problem is utilizing the basic geometry information to infer the GNSS receiver location. Since we have multiple satellite, we can optimize the location to achieve the minimum projection error among all satellite based by least square.

To be more specific,

$$r_u^k = ||x^k_{sat} - x_u||$$

where,

$r_u^k$: the actually distance between user device (GNSS Receiver) to Satellite;

$x^k_{sat}$: the actually position of Satellite;

$x_u$: the actually position of user device (GNSS Receiver).

$$ Pr_u^k = r_u^k + b_u + \sigma^k $$

where,

$Pr_u^k$: the corrected pseudorange (i.e. a closer approximation to the geometric range from the phone to the satellite) can be computed as: $correctedPrM = rawPrM + satClkBiasM - isrbM - ionoDelayM - tropoDelayM$;

$b_u$: the user clock bias in meters equivalent, which $b_u = clock\_bias_u * LIGHTSPEED$;

$\sigma^k$: the actually position of user device (GNSS Receiver).

Our aim is to minimize the measured and the estimated pseudorange for each satellite:

$$\delta Pr_u^k = Pr_u^k - (r_u^k + b_u) = Pr_u^k - (||x^k_{sat} - x_u|| + b_u)$$

which is our target function in least square solver.

<!-- Which can be deducted to based equation (5) in [here](https://www.telesens.co/2017/07/17/calculating-position-from-raw-gps-data/):

$$\delta Pr_u^k = - \frac{x^k_{sat} - x_u}{||x^k_{sat} - x_u||} \delta x_u^k + \delta b_u + \sigma^k = - \hat{x}_u^k \delta x_u^k + \delta b_u + \sigma^k$$ -->

Let's try it with 1 single epoch.

In [ ]:
epoch_time = 1273529464442
df_sample_epoch = df_sample_trail[df_sample_trail.millisSinceGpsEpoch == epoch_time]
df_sample_epoch_gt = df_sample_trail_gt[df_sample_trail_gt.millisSinceGpsEpoch == epoch_time]
df_sample_epoch_baseline = df_baseline[(df_baseline.collectionName == "2020-05-14-US-MTV-1") & (df_baseline.phoneName == "Pixel4") & (df_baseline.millisSinceGpsEpoch == epoch_time)]

sat_pos = df_sample_epoch[["xSatPosM","ySatPosM","zSatPosM"]].to_numpy()
pseudoranges = np.squeeze(df_sample_epoch[["correctedPrM"]].to_numpy())

### self implement simple least square

Build the simplest version from [code](https://www.kaggle.com/realbigmitchell/simple-least-squares-solution) and [article](https://www.telesens.co/2017/07/17/calculating-position-from-raw-gps-data/).

In [ ]:
def least_squares(sat_pos, pseudoranges, weights=1, x_hat=np.array([0, 0, 0, 0])):
  """
  Args:
    sat_pos: The satellite position (meters) in an ECEF coordinate frame
    pseudoranges: The corrected pseudorange (i.e. a closer approximation to the geometric range from the phone to the satellite)
    x_hat: the phone's initial/previous estimated position (x, y, z, b) and 
           b represent the user clock bias in units of distance = clock bias (t) * light speed (c)
  
  Returns:
    x_hat: the phone's estimated position
    norm_dp:
  """
  dx = np.Inf*np.ones(3)
  G = np.ones((pseudoranges.size, 4))
  iterations = 0

  if isinstance(weights, np.ndarray):
    weights = np.diag(weights)
  else:
    weights = weights*np.eye(pseudoranges.size)

  while np.linalg.norm(dx) > 1e-3:
    norms = np.linalg.norm(sat_pos - x_hat[:3], axis=1)
    dp = pseudoranges - norms - x_hat[3]
    G[:, 0:3] = -(sat_pos - x_hat[:3])/norms[:, None]
    # G_T = np.transpose(G)
    # dx = np.linalg.inv(G_T@G) @ G_T @ dp
    dx = np.linalg.pinv(weights@G) @ weights @ dp
    x_hat = x_hat + dx
    iterations += 1
  return x_hat, np.linalg.norm(dp)

In [ ]:
x, dp = least_squares(sat_pos, pseudoranges)

print("Ground truth:", df_sample_epoch_gt[["latDeg","lngDeg","heightAboveWgs84EllipsoidM"]].to_numpy())
print("Weighted Least Square Estimation (Baseline):", df_sample_epoch_baseline[["latDeg","lngDeg","heightAboveWgs84EllipsoidM"]].to_numpy())
print("Simple Least Square Estimation:", *ecef2lla(*x[:3]))
# print(b/LIGHTSPEED)
# print(dp)

Ground truth: [[  37.42357595 -122.09413204   33.21      ]]
Weighted Least Square Estimation (Baseline): [[  37.4235777 -122.094101   -33.29     ]]
Simple Least Square Estimation: [[37.42361372]] [[-122.0936959]] [[-26.18615617]]


### scipy optimize version

Build the simplest version from [this code](https://github.com/commaai/laika/blob/master/laika/raw_gnss.py#L254-L350)

In [ ]:
import scipy.optimize as opt

def calc_pos_fix(sat_pos, pr, weights=1, x0=[0, 0, 0, 0]):
  '''
  Calculates gps fix with WLS optimizer
  returns:
  0 -> list with positions
  1 -> pseudorange errs
  '''
  n = len(pr)
  if n < 3:
      return x0, []

  Fx_pos = pr_residual(sat_pos, pr, weights=weights)
  opt_pos = opt.least_squares(Fx_pos, x0).x
  return opt_pos, Fx_pos(opt_pos, weights=1)


def pr_residual(sat_pos, pr, weights=1):
  # solve for pos
  def Fx_pos(x_hat, weights=weights):
    rows = weights * (np.linalg.norm(sat_pos - x_hat[:3], axis=1) + x_hat[3] - pr)
    return rows
  return Fx_pos

In [ ]:
x, dp = calc_pos_fix(sat_pos, pseudoranges)

print("Ground truth:", df_sample_epoch_gt[["latDeg","lngDeg","heightAboveWgs84EllipsoidM"]].to_numpy())
print("Weighted Least Square Estimation (Baseline):", df_sample_epoch_baseline[["latDeg","lngDeg","heightAboveWgs84EllipsoidM"]].to_numpy())
print("Simple Least Square Estimation:", *ecef2lla(*x[:3]))
# print(x[3]/LIGHTSPEED)
# print(dp)

Ground truth: [[  37.42357595 -122.09413204   33.21      ]]
Weighted Least Square Estimation (Baseline): [[  37.4235777 -122.094101   -33.29     ]]
Simple Least Square Estimation: [[37.42361372]] [[-122.0936959]] [[-26.18615579]]


## Weighted Least Square Solution

The signal from each satellite may not that reliable. In this case, we can apply the reliability of the signal as weights for each signal to improve the least square result. Here, we use the `rawPrUncM` as the weights for signal, which represents the pseudorange uncertainty/deviation of each signal. The relative code can be found at [the google official repo](WLS shown in official repo](https://github.com/google/gps-measurement-tools/blob/master/opensource/WlsPvt.m#L62-L111)

In [ ]:
pseudoranges_sigma = np.squeeze(df_sample_epoch[["rawPrUncM"]].to_numpy())

x, dp = least_squares(sat_pos, pseudoranges, 1/pseudoranges_sigma)

print("Ground truth:", df_sample_epoch_gt[["latDeg","lngDeg","heightAboveWgs84EllipsoidM"]].to_numpy())
print("Weighted Least Square Estimation (Baseline):", df_sample_epoch_baseline[["latDeg","lngDeg","heightAboveWgs84EllipsoidM"]].to_numpy())
print("Weighted Least Square Estimation:", *ecef2lla(*x[:3]))

Ground truth: [[  37.42357595 -122.09413204   33.21      ]]
Weighted Least Square Estimation (Baseline): [[  37.4235777 -122.094101   -33.29     ]]
Weighted Least Square Estimation: [[37.42357917]] [[-122.09375829]] [[-35.25074978]]


In [ ]:
pseudoranges_sigma = np.squeeze(df_sample_epoch[["rawPrUncM"]].to_numpy())

x, dp = calc_pos_fix(sat_pos, pseudoranges, 1/pseudoranges_sigma)

print("Ground truth:", df_sample_epoch_gt[["latDeg","lngDeg","heightAboveWgs84EllipsoidM"]].to_numpy())
print("Weighted Least Square Estimation (Baseline):", df_sample_epoch_baseline[["latDeg","lngDeg","heightAboveWgs84EllipsoidM"]].to_numpy())
print("Weighted Least Square Estimation:", *ecef2lla(*x[:3]))

Ground truth: [[  37.42357595 -122.09413204   33.21      ]]
Weighted Least Square Estimation (Baseline): [[  37.4235777 -122.094101   -33.29     ]]
Weighted Least Square Estimation: [[37.42357917]] [[-122.09375829]] [[-35.25075018]]


## Evaluation

In [ ]:
def calc_haversine(lat1, lon1, lat2, lon2):
  """Calculates the great circle distance between two points
  on the earth. Inputs are array-like and specified in decimal degrees.
  """
  RADIUS = 6_367_000
  lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
  dlat = lat2 - lat1
  dlon = lon2 - lon1
  a = np.sin(dlat/2)**2 + \
      np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
  dist = 2 * RADIUS * np.arcsin(a**0.5)
  return dist

### Evaluate in one epoch

In [ ]:
deg_gt = df_sample_epoch_gt[["latDeg","lngDeg"]].to_numpy().transpose()
deg_baseline = df_sample_epoch_baseline[["latDeg","lngDeg"]].to_numpy().transpose()
print("Weighted Least Square (baseline) haversine distance (M):", calc_haversine(*deg_gt, *deg_baseline))
print("Weighted Least Square haversine distance (M):", calc_haversine(*deg_gt, *ecef2lla(*x[:3])[:2]))

Weighted Least Square (baseline) haversine distance (M): [2.74590055]
Weighted Least Square haversine distance (M): [[32.98570796]]


### Evaluate in whole training

In [ ]:
def simple_pipeline(df_trails):
  """ simple pipeline to estimate the GNSS receiver location by least square
  Args:
    df_trails: the df read from derived file
  
  Returns:
    result df with estimated degrees and heights
  """
  df_trails["correctedPrM"] = df_trails["rawPrM"] + df_trails["satClkBiasM"] - df_trails["isrbM"] - df_trails["ionoDelayM"] - df_trails["tropoDelayM"]
  
  results = []
  x = [0, 0, 0, 0]
  df_epochs = df_trails.groupby(["collectionName", "phoneName", "millisSinceGpsEpoch"])
  for indices, df_epoch in tqdm(df_epochs, desc="Estimate location by LS for epoch"):
    sat_pos = df_epoch[["xSatPosM","ySatPosM","zSatPosM"]].to_numpy()
    pseudoranges = np.squeeze(df_epoch[["correctedPrM"]].to_numpy())
    pseudoranges_sigma = np.squeeze(df_epoch[["rawPrUncM"]].to_numpy())
    x, _ = calc_pos_fix(sat_pos, pseudoranges, 1/pseudoranges_sigma, x)
    # x, _ = calc_pos_fix(sat_pos, pseudoranges, 1, x)
    values = np.squeeze(ecef2lla(*x[:3]))
    results.append([*indices, *values])
  return pd.DataFrame(results,columns=["collectionName", "phoneName", "millisSinceGpsEpoch", "latDeg", "lngDeg", "heightAboveWgs84EllipsoidM"])

In [ ]:
from pathlib import Path
# from tqdm import tqdm
from tqdm.notebook import tqdm

# datapath = Path("./data")
ground_truths = (datapath / "train").rglob("ground_truth.csv")
drived_files = (datapath / "train").rglob("*_derived.csv")

df_sample_trails_baseline = pd.read_csv(datapath / 'baseline_locations_train.csv')
df_sample_trails_gt = pd.concat([pd.read_csv(filepath) for filepath in tqdm(ground_truths, total=73, desc="Reading ground truth data")], ignore_index=True)
df_sample_trails = pd.concat([pd.read_csv(filepath) for filepath in tqdm(drived_files, total=73, desc="Reading drived data")], ignore_index=True)

Reading ground truth data:   0%|          | 0/73 [00:00<?, ?it/s]

Reading drived data:   0%|          | 0/73 [00:00<?, ?it/s]

In [ ]:
df_sample_trails_gt["receivedSvTimeInGpsNanos"] = df_sample_trails_gt.millisSinceGpsEpoch*int(1e6)
df_sample_trails_raw = df_sample_trails.drop("millisSinceGpsEpoch", axis=1)

df_merge = pd.merge_asof(df_sample_trails_raw.sort_values('receivedSvTimeInGpsNanos'), df_sample_trails_gt.sort_values('receivedSvTimeInGpsNanos'), 
                                           on="receivedSvTimeInGpsNanos", by=["collectionName", "phoneName"], direction='nearest',tolerance=int(1e9))
df_merge = df_merge.sort_values(by=["collectionName", "phoneName", "millisSinceGpsEpoch"], ignore_index=True)

In [ ]:
df_sample_trails_estimate = simple_pipeline(df_merge)

Estimate location by LS for epoch:   0%|          | 0/130756 [00:00<?, ?it/s]

In [ ]:
df_sample_trails_merged_baseline = pd.merge_asof(df_sample_trails_gt.sort_values('millisSinceGpsEpoch'),
                                                 df_sample_trails_baseline.sort_values('millisSinceGpsEpoch'), 
                                                 on="millisSinceGpsEpoch", by=["collectionName", "phoneName"], 
                                                 direction='nearest',tolerance=100000, suffixes=('_truth', '_pred'))
df_sample_trails_merged_baseline = df_sample_trails_merged_baseline.sort_values(by=["collectionName", "phoneName", "millisSinceGpsEpoch"], ignore_index=True)

df_sample_trails_merged_SL = pd.merge_asof(df_sample_trails_gt.sort_values('millisSinceGpsEpoch'), 
                                           df_sample_trails_estimate.sort_values('millisSinceGpsEpoch'), 
                                           on="millisSinceGpsEpoch", by=["collectionName", "phoneName"], 
                                           direction='nearest',tolerance=100000, suffixes=('_truth', '_pred'))
df_sample_trails_merged_SL = df_sample_trails_merged_SL.sort_values(by=["collectionName", "phoneName", "millisSinceGpsEpoch"], ignore_index=True)

compared_cols = ["latDeg_truth","lngDeg_truth","latDeg_pred","lngDeg_pred"]
print("Weighted Least Square (baseline) haversine distance (M):", calc_haversine(*df_sample_trails_merged_baseline[compared_cols].to_numpy().transpose()).mean())
print("Weighted Least Square haversine distance (M):", calc_haversine(*df_sample_trails_merged_SL[compared_cols].to_numpy().transpose()).mean())

Weighted Least Square (baseline) haversine distance (M): 3.8468483749952074
Weighted Least Square haversine distance (M): 30.854244628290758


## Submission

It should be noticed that the epoch time in the drived file and submission/baseline file is not same, in this case, we need to merge it to submission format with epoch time tolerant and sort it into suitable output format (based on quick experiment, 100s tolerant can be merged without empty value).

In [ ]:
from pathlib import Path
# from tqdm import tqdm
from tqdm.notebook import tqdm

# datapath = Path("./data")
drived_files = (datapath / "test").rglob("*_derived.csv")

df_sample_trails_baseline = pd.read_csv(datapath / 'baseline_locations_test.csv')
df_sample_trails = pd.concat([pd.read_csv(filepath) for filepath in tqdm(drived_files, total=48, desc="Reading drived data")], ignore_index=True)

Reading drived data:   0%|          | 0/48 [00:00<?, ?it/s]

In [ ]:
df_sample_trails_baseline["receivedSvTimeInGpsNanos"] = df_sample_trails_baseline.millisSinceGpsEpoch*int(1e6)
df_sample_trails_raw = df_sample_trails.drop("millisSinceGpsEpoch", axis=1)

df_merge = pd.merge_asof(df_sample_trails_raw.sort_values('receivedSvTimeInGpsNanos'), df_sample_trails_baseline.sort_values('receivedSvTimeInGpsNanos'), 
                                           on="receivedSvTimeInGpsNanos", by=["collectionName", "phoneName"], direction='nearest',tolerance=int(1e9))
df_merge = df_merge.sort_values(by=["collectionName", "phoneName", "millisSinceGpsEpoch"], ignore_index=True)

In [ ]:
df_sample_trails_estimate = simple_pipeline(df_merge)

Estimate location by LS for epoch:   0%|          | 0/90886 [00:00<?, ?it/s]

In [ ]:
df_sample_trails_baseline = df_sample_trails_baseline.drop(["latDeg","lngDeg","heightAboveWgs84EllipsoidM"], axis=1)
df_sample_trails_merged = pd.merge_asof(df_sample_trails_baseline.sort_values('millisSinceGpsEpoch'), 
                                        df_sample_trails_estimate.sort_values('millisSinceGpsEpoch'), 
                                        on="millisSinceGpsEpoch", by=["collectionName", "phoneName"], direction='nearest', tolerance=100000)
df_sample_trails_merged = df_sample_trails_merged.sort_values(by=["phone", "millisSinceGpsEpoch"], ignore_index=True)

df_submission = df_sample_trails_merged[["phone", "millisSinceGpsEpoch", "latDeg", "lngDeg"]].copy()
df_submission.to_csv('submission.csv', index=False)